<a href="https://colab.research.google.com/github/ShangWeiKuo/106-2-R-project/blob/master/Code/ResearchMethod/health/3day/LSTM_Bootstrap_health_w3day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tsmoothie
!pip install keras-hypetune

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tsmoothie.smoother import *
from tsmoothie.utils_func import create_windows

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
from keras.optimizers import Adam
from kerashypetune import KerasGridSearch
from pandas import read_csv
from pandas import DataFrame
import numpy as np
import pickle

In [ ]:
window_shape = 3
time_h = 1

In [ ]:
import rpy2 
%load_ext rpy2.ipython

In [ ]:
%%R
require(readr)
require(dplyr)
df = read_csv("/content/drive/MyDrive/paper/raw_covid_data.csv")

R[write to console]: Loading required package: readr

R[write to console]: Loading required package: dplyr

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





── Column specification ────────────────────────────────────────────────────────
cols(
  positive = col_double(),
  death = col_double(),
  new_test = col_double(),
  new_case = col_double(),
  dis_less_one = col_double(),
  dis_one_twtf = col_double(),
  dis_twtf_hum = col_double(),
  dis_higher_hum = col_double(),
  passenger_d_volume = col_double(),
  passenger_i_volume = col_double(),
  person_screened_number = col_double()
)



In [ ]:
%%R
install.packages("forecast"); library(forecast)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =


In [ ]:
sample_num = 51

In [ ]:
%%R
sample_num = 51
ts_df <- matrix(1:4554, ncol = 11, nrow = 414)
for(i in c(1:ncol(df))){
  ts_df[, i] = ts(df[, i], frequency=1)
}
ts_df <- ts_df %>% data.frame()
train <- ts_df[1:331, 1:4]
test <- ts_df[332:414, 1:4]

colnames(train) <- colnames(df)[1:4]
colnames(test) <- colnames(df)[1:4]

cols = colnames(train)

### BOOTSTRAP ###
for(i in c(1:4)){
  if(i == 1){
    train_boot = bld.mbb.bootstrap(train[, i], sample_num, block_size = 3) %>% data.frame()
  }else{
    temp_boot = bld.mbb.bootstrap(train[, i], sample_num, block_size = 3) %>% data.frame()
    train_boot = cbind(train_boot, temp_boot)
  }
}
train_boot_df <- train_boot %>% data.frame()

In [ ]:
train_boot_df=%R train_boot_df
train=%R train
test=%R test
cols=%R cols

In [ ]:
bts_2d = []
bts_3d = []

In [ ]:
for i in range(4):
  for j in range(1, sample_num):
    bts_3d.append(train_boot_df.iloc[:, j+(sample_num*i)])
  bts_2d.append(bts_3d)
  bts_3d = []

bts_2d = np.stack(bts_2d, axis=-1)
bts_2d.shape

(50, 331, 4)

In [ ]:
bts_2d_y = bts_2d[:, :, 3].reshape((bts_2d.shape[0], bts_2d.shape[1], 1))
train_y = pd.DataFrame(train['new_case'])
test_y = pd.DataFrame(test['new_case'])

In [ ]:
### CREATE TRAIN/VALIDATION WINDOWS W/ BOOTSTRAP ###

_X = create_windows(train[cols], window_shape=window_shape, end_id=-time_h)
_y = create_windows(train_y, window_shape=time_h, start_id=window_shape)
_X_train, _X_val, _y_train, _y_val = train_test_split(_X, _y, test_size=window_shape*22, shuffle=True)
X_train, X_val, y_train, y_val = [_X_train], [_X_val], [_y_train], [_y_val]

for i in range(1, bts_2d.shape[0]):

    _X = create_windows(bts_2d[i], window_shape=window_shape, end_id=-time_h)
    _y = create_windows(bts_2d_y[i], window_shape=time_h, start_id=window_shape)
    _X_train, _X_val, _y_train, _y_val = train_test_split(_X, _y, test_size=window_shape*22, shuffle=True)
    X_train.append(_X_train); X_val.append(_X_val)
    y_train.append(_y_train); y_val.append(_y_val)
    
X_train = np.concatenate(X_train); X_val = np.concatenate(X_val)
y_train = np.concatenate(y_train); y_val = np.concatenate(y_val)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(13100, 3, 4) (13100, 1, 1)
(3300, 3, 4) (3300, 1, 1)


In [ ]:
Min_train = []
Min_val = []
Max_train = []
Max_val = []

In [ ]:
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[2]): 
    Min_train.append(min(X_train[i, :, j]))
  np.vstack(Min_train)

Min_train = np.array(Min_train)
Min_train = Min_train.reshape(X_train.shape[0], 1, X_train.shape[2])

In [ ]:
for i in range(X_val.shape[0]):
  for j in range(X_val.shape[2]): 
    Min_val.append(min(X_val[i, :, j]))
  np.vstack(Min_val)

Min_val = np.array(Min_val)
Min_val = Min_val.reshape(X_val.shape[0], 1, X_val.shape[2])

In [ ]:
Min_train.shape
Min_val.shape

(3300, 1, 4)

In [ ]:
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[2]): 
    Max_train.append(max(X_train[i, :, j]))
  np.vstack(Max_train)

Max_train = np.array(Max_train)
Max_train = Max_train.reshape(X_train.shape[0], 1, X_train.shape[2])

In [ ]:
for i in range(X_val.shape[0]):
  for j in range(X_val.shape[2]): 
    Max_val.append(max(X_val[i, :, j]))
  np.vstack(Max_val)

Max_val = np.array(Max_val)
Max_val = Max_val.reshape(X_val.shape[0], 1, X_val.shape[2])

In [ ]:
Max_train.shape
Max_val.shape

(3300, 1, 4)

In [ ]:
y_train = (y_train[...,[-1]] - Min_train[...,[-1]]) / (Max_train[...,[-1]] - Min_train[...,[-1]])
X_train = (X_train - Min_train) / (Max_train - Min_train)

y_val = (y_val[...,[-1]] - Min_val[...,[-1]]) / (Max_val[...,[-1]] - Min_val[...,[-1]])
X_val = (X_val - Min_val) / (Max_val - Min_val)

In [ ]:
### CREATE TEST WINDOWS ###

X_test = create_windows(test[cols], window_shape=window_shape, end_id=-time_h)
y_test = create_windows(test_y, window_shape=time_h, start_id=window_shape)

In [ ]:
Min_test = []
Max_test = []

In [ ]:
for i in range(X_test.shape[0]):
  for j in range(X_test.shape[2]): 
    Min_test.append(min(X_test[i, :, j]))
  np.vstack(Min_test)

Min_test = np.array(Min_test)
Min_test = Min_test.reshape(X_test.shape[0], 1, X_test.shape[2])

In [ ]:
for i in range(X_test.shape[0]):
  for j in range(X_test.shape[2]): 
    Max_test.append(max(X_test[i, :, j]))
  np.vstack(Max_test)

Max_test = np.array(Max_test)
Max_test = Max_test.reshape(X_test.shape[0], 1, X_test.shape[2])

In [ ]:
X_test = (X_test - Min_test) / (Max_test - Min_test)
y_test = np.squeeze(y_test[...,[-1]], -1)

X_test.shape, y_test.shape

((80, 3, 4), (80, 1))

In [ ]:
def get_model_mse(param):
    
    inp = Input((window_shape,len(cols)))
    enc = LSTM(param['lstm_unit'], activation=param['lstm_act'], 
               return_sequences=False)(inp)
    x = RepeatVector(time_h)(enc)
    
    out = Dense(1)(x)
    
    model = Model(inp, out)
    model.compile(Adam(lr=param['lr']), 'mse')
    
    return model

In [ ]:
param_grid = {
    'lstm_unit': [128,64], 
    'lstm_act': ['tanh'],
    'lr': [1e-4,1e-3], 
    'epochs': 100, 
    'batch_size': [16,32,128]
}

es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_loss', mode='auto', restore_best_weights=True)
hypermodel_mse = get_model_mse
kgs_mse = KerasGridSearch(hypermodel_mse, param_grid, monitor='val_loss', greater_is_better=False, tuner_verbose=1)
kgs_mse.search(X_train, y_train, validation_data=(X_val, y_val), callbacks=[es])


12 trials detected for ('lstm_unit', 'lstm_act', 'lr', 'epochs', 'batch_size')


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")



***** (1/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 16})
SCORE: 2.4776 at epoch 100

***** (2/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 32})
SCORE: 2.51038 at epoch 100

***** (3/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 128})
SCORE: 2.56514 at epoch 100

***** (4/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 16})
SCORE: 2.38115 at epoch 97

***** (5/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 32})
SCORE: 2.38502 at epoch 100

***** (6/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 128})
SCORE: 2.4067 at epoch 100

***** (7/12) *****
Search({'lstm_unit': 64, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 16})
SCORE: 2.49306 at epoch 100

***** (8/1

<kerashypetune.KerasGridSearch>

In [ ]:
pred_boot_mse = kgs_mse.best_model.predict(X_test)
pred_boot_mse

array([[[ 0.85669374]],

       [[ 0.23435351]],

       [[-0.16734967]],

       [[-0.13851397]],

       [[ 0.24573132]],

       [[ 1.3394113 ]],

       [[ 0.68009377]],

       [[ 0.7487875 ]],

       [[ 0.46420735]],

       [[-0.1218864 ]],

       [[-0.25433007]],

       [[-0.19119245]],

       [[ 1.2806112 ]],

       [[ 1.3584063 ]],

       [[ 0.6628325 ]],

       [[ 0.04713486]],

       [[-0.27016327]],

       [[-0.20938328]],

       [[-0.10679021]],

       [[ 1.065392  ]],

       [[ 1.2650342 ]],

       [[ 1.1792496 ]],

       [[ 0.5065727 ]],

       [[-0.17100981]],

       [[-0.15685526]],

       [[ 0.18841635]],

       [[ 1.292865  ]],

       [[ 1.4332685 ]],

       [[ 0.69192535]],

       [[-0.06118432]],

       [[-0.1672003 ]],

       [[-0.16375631]],

       [[ 0.21080178]],

       [[ 1.2730932 ]],

       [[ 1.2526493 ]],

       [[ 0.60342073]],

       [[ 0.14045377]],

       [[-0.08625948]],

       [[-0.21331456]],

       [[ 0.03369093]],



In [ ]:
pred_boot_mse = np.squeeze(pred_boot_mse, -1)

In [ ]:
pred_boot_mse = ((Max_test[:, :, 3] - Min_test[:, :, 3]) * pred_boot_mse) + Min_test[:, :, 3]

In [ ]:
pd.DataFrame(pred_boot_mse).to_csv("/content/drive/MyDrive/paper/health/PredictionData_boot_health_w3day_mse.csv")
pd.DataFrame(y_test).to_csv("/content/drive/MyDrive/paper/health/TrueData_boot_health_w3day.csv")

In [ ]:
def get_model_mae(param):
    
    inp = Input((window_shape,len(cols)))
    enc = LSTM(param['lstm_unit'], activation=param['lstm_act'], 
               return_sequences=False)(inp)
    x = RepeatVector(time_h)(enc)
    
    out = Dense(1)(x)
    
    model = Model(inp, out)
    model.compile(Adam(lr=param['lr']), 'mae')
    
    return model

In [ ]:
hypermodel_mae = get_model_mae
kgs_mae = KerasGridSearch(hypermodel_mae, param_grid, monitor='val_loss', greater_is_better=False, tuner_verbose=1)
kgs_mae.search(X_train, y_train, validation_data=(X_val, y_val), callbacks=[es])


12 trials detected for ('lstm_unit', 'lstm_act', 'lr', 'epochs', 'batch_size')

***** (1/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 16})


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


SCORE: 0.91847 at epoch 100

***** (2/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 32})
SCORE: 0.92435 at epoch 100

***** (3/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.0001, 'epochs': 100, 'batch_size': 128})
SCORE: 0.95114 at epoch 100

***** (4/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 16})
Restoring model weights from the end of the best epoch.
Epoch 00049: early stopping
SCORE: 0.91016 at epoch 49

***** (5/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 32})
Restoring model weights from the end of the best epoch.
Epoch 00070: early stopping
SCORE: 0.91167 at epoch 70

***** (6/12) *****
Search({'lstm_unit': 128, 'lstm_act': 'tanh', 'lr': 0.001, 'epochs': 100, 'batch_size': 128})
Restoring model weights from the end of the best epoch.
Epoch 00076: early stopping
SCORE: 0.91418 at epoch 76

***** (7/12) 

<kerashypetune.KerasGridSearch>

In [ ]:
pred_boot_mae = kgs_mae.best_model.predict(X_test)
pred_boot_mae

array([[[ 0.96021134]],

       [[ 0.35687274]],

       [[ 0.13333248]],

       [[ 0.10841422]],

       [[ 0.3520019 ]],

       [[ 1.1336124 ]],

       [[ 0.5154798 ]],

       [[ 0.5410745 ]],

       [[ 0.5917339 ]],

       [[ 0.14362906]],

       [[-0.01624044]],

       [[-0.03862163]],

       [[ 1.0156362 ]],

       [[ 1.2348374 ]],

       [[ 0.7462013 ]],

       [[ 0.22021362]],

       [[ 0.00630273]],

       [[ 0.04117056]],

       [[ 0.03564389]],

       [[ 0.8074809 ]],

       [[ 1.0696121 ]],

       [[ 1.0726005 ]],

       [[ 0.55640656]],

       [[ 0.08035687]],

       [[ 0.0881509 ]],

       [[ 0.29355997]],

       [[ 1.0793962 ]],

       [[ 1.2862637 ]],

       [[ 0.69743073]],

       [[ 0.10928947]],

       [[ 0.0995107 ]],

       [[ 0.08632605]],

       [[ 0.2577685 ]],

       [[ 1.0645189 ]],

       [[ 1.1319528 ]],

       [[ 0.781264  ]],

       [[ 0.24840714]],

       [[ 0.13662858]],

       [[ 0.03164851]],

       [[ 0.14707936]],



In [ ]:
pred_boot_mae = np.squeeze(pred_boot_mae, -1)

In [ ]:
pred_boot_mae = ((Max_test[:, :, 3] - Min_test[:, :, 3]) * pred_boot_mae) + Min_test[:, :, 3]

In [ ]:
pd.DataFrame(pred_boot_mae).to_csv("/content/drive/MyDrive/paper/health/PredictionData_boot_health_w3day_mae.csv")

In [ ]:
kgs_mae.best_model.save("/content/drive/MyDrive/paper/feature_importance/health/Data_augmentation/w3day")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/paper/feature_importance/health/Data_augmentation/w3day/assets


In [ ]:
# import pickle
with open('/content/drive/MyDrive/paper/feature_importance/health/Data_augmentation/w3day/X_test_health_w3day.pkl', 'wb') as f:
    pickle.dump(X_test, f)
with open('/content/drive/MyDrive/paper/feature_importance/health/Data_augmentation/w3day/cols_health_w3day.pkl', 'wb') as f:
    pickle.dump(cols, f)